In [1]:
import os, pathlib
os.environ["MPLCONFIGDIR"] = str(pathlib.Path.cwd() / ".mpl-cache")

print("MPLCONFIGDIR set to", os.environ["MPLCONFIGDIR"])


MPLCONFIGDIR set to /Users/ethan/Documents/GitHub/airline-revenue-analytics/notebooks/booking/.mpl-cache


In [2]:
# Project paths (booking pipeline)
from pathlib import Path
import sys

def find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / "pyproject.toml").exists() and (p / "src" / "airline_revenue_analytics").exists():
            return p
    return start

REPO_ROOT = find_repo_root(Path.cwd())
PROJECT_ROOT = REPO_ROOT
SRC_ROOT = REPO_ROOT / "src"
if str(SRC_ROOT) not in sys.path:
    sys.path.append(str(SRC_ROOT))

from airline_revenue_analytics.config import get_paths

PATHS = get_paths("booking")
DATA_DIR = REPO_ROOT / "data"
RAW_DIR = PATHS.data_raw
DB_PATH = PATHS.db_path
OUTPUT_DIR = PATHS.outputs_root
FIG_DIR = PATHS.figures
TAB_DIR = PATHS.tables
ART_DIR = PATHS.artifacts

def _rel(p: Path) -> str:
    try:
        return str(Path(p).resolve().relative_to(REPO_ROOT))
    except Exception:
        return Path(p).name

print("REPO_ROOT:", REPO_ROOT.name)
print("DB_PATH:", _rel(DB_PATH))
print("OUTPUT_DIR:", _rel(OUTPUT_DIR))


REPO_ROOT: airline-revenue-analytics
DB_PATH: data/raw/airlines_db.sqlite
OUTPUT_DIR: outputs/booking


 
# 02 - Data Preparation (Steps 3.1–3.5)
Goal: select/clean/construct/integrate/format to a booking-level modeling table.
Outputs:
- `outputs/booking/tables/seg_master_sample.csv` (evidence)
- `outputs/booking/tables/booking_model_df.parquet` (modeling dataset)
- `outputs/booking/tables/booking_model_df_preview.csv` & `table_3_5_summary.csv`


In [3]:
# Imports & path setup
import sys, pathlib, numpy as np, pandas as pd
import matplotlib; import matplotlib.pyplot as plt

# Ensure repository root on sys.path (so "src" is importable when running from notebooks/)
sys.path.append(str(PROJECT_ROOT))

from airline_revenue_analytics.io import find_sqlite, load_core_tables, replace_literal_N
from airline_revenue_analytics.features.segment import to_utc, minutes_between, days_between  # we'll engineer features inline here

# Robust RAW_DIR detection (works if CWD is notebooks/)
CANDIDATE_RAW_DIRS = [RAW_DIR]
RAW_DIR = next((p.resolve() for p in CANDIDATE_RAW_DIRS if p.exists()), CANDIDATE_RAW_DIRS[0].resolve())

OUT_DIR = OUTPUT_DIR
FIG_DIR = OUT_DIR / "figures"; FIG_DIR.mkdir(parents=True, exist_ok=True)
TAB_DIR = OUT_DIR / "tables";  TAB_DIR.mkdir(parents=True, exist_ok=True)

print("RAW_DIR:", RAW_DIR)
print("OUT_DIR:", _rel(OUT_DIR))


RAW_DIR: /Users/ethan/Documents/GitHub/airline-revenue-analytics/data/raw
OUT_DIR: outputs/booking


In [4]:
# Load core tables and normalise '\N' to NaN
db_path = find_sqlite(RAW_DIR)
print("Using DB:", db_path)

tables = load_core_tables(db_path)
for k in list(tables):
    tables[k] = replace_literal_N(tables[k])

print("Loaded tables:", list(tables.keys()))


Using DB: /Users/ethan/Documents/GitHub/airline-revenue-analytics/data/raw/airlines_db.sqlite
Loaded tables: ['bookings', 'tickets', 'ticket_flights', 'flights', 'boarding_passes', 'seats', 'aircrafts_data', 'airports_data']


In [5]:
from airline_revenue_analytics.viz.charts import apply_style, PLOT_COLORS
apply_style()
PASS_COLOR = "#D9F2E6"
FAIL_COLOR = "#FCE4E4"
NEG_BG_COLOR = FAIL_COLOR


In [6]:
# Keep only ARRIVED flights to avoid cancelled/unknown segments
fl = tables["flights"].copy()
fl["status"] = fl["status"].astype(str).str.upper()
fl = fl[fl["status"] == "ARRIVED"].copy()

# Join ticket_flights with flights (route + schedule)
tf = tables["ticket_flights"].copy()
tf["amount"] = pd.to_numeric(tf["amount"], errors="coerce")

keep_cols = ["flight_id","scheduled_departure","scheduled_arrival","departure_airport","arrival_airport"]
tf_fl = tf.merge(fl[keep_cols], on="flight_id", how="inner", validate="many_to_one")

# Remove non-positive prices (quality gate)
tf_fl = tf_fl[tf_fl["amount"] > 0].copy()

# Evidence table for selection
sel_counts = pd.DataFrame({
    "stage": ["ticket_flights_raw","arrived_join","positive_amount"],
    "rows":  [len(tf),            len(tf.merge(fl[["flight_id"]], on="flight_id", how="inner")), len(tf_fl)]
})
sel_counts.to_csv(TAB_DIR/"table_3_1_selection_counts.csv", index=False)
sel_counts


,stage,rows
0,ticket_flights_raw,1045726
1,arrived_join,574830
2,positive_amount,574830


In [7]:
# Key types and timestamp harmonisation (schedule-based; avoid leakage)
tf_fl["flight_id"] = pd.to_numeric(tf_fl["flight_id"], errors="coerce").astype("Int64")
tf_fl["scheduled_departure"] = to_utc(tf_fl["scheduled_departure"])
tf_fl["scheduled_arrival"]   = to_utc(tf_fl["scheduled_arrival"])

# Fare class normalisation
tf_fl["fare_conditions"] = tf_fl["fare_conditions"].astype("string").str.strip().str.title()

# Build route code
tf_fl = tf_fl.rename(columns={"departure_airport":"dep_airport","arrival_airport":"arr_airport"})
tf_fl["route_code"] = tf_fl["dep_airport"].astype(str) + "-" + tf_fl["arr_airport"].astype(str)

tf_fl.head(3)


,ticket_no,flight_id,fare_conditions,amount,scheduled_departure,scheduled_arrival,dep_airport,arr_airport,route_code
0,0005432159776,30625,Business,42100,2017-07-16 15:15:00+00:00,2017-07-16 17:00:00+00:00,AER,SVO,AER-SVO
1,0005435212351,30625,Business,42100,2017-07-16 15:15:00+00:00,2017-07-16 17:00:00+00:00,AER,SVO,AER-SVO
2,0005435212386,30625,Business,42100,2017-07-16 15:15:00+00:00,2017-07-16 17:00:00+00:00,AER,SVO,AER-SVO


In [8]:
# Segment-level engineered features (schedule only)
tf_fl["sched_flight_duration_minutes"] = minutes_between(tf_fl["scheduled_departure"], tf_fl["scheduled_arrival"])
tf_fl["departure_dow"]  = tf_fl["scheduled_departure"].dt.dayofweek
tf_fl["departure_hour"] = tf_fl["scheduled_departure"].dt.hour
tf_fl["fare_class"]     = tf_fl["fare_conditions"]
tf_fl["fare_class_ord"] = tf_fl["fare_class"].map({"Economy":0,"Comfort":1,"Business":2}).astype("Int64")
tf_fl["is_premium_cabin"]= tf_fl["fare_class"].isin(["Comfort","Business"]).astype(int)

tf_fl[["sched_flight_duration_minutes","departure_dow","departure_hour","fare_class","fare_class_ord","is_premium_cabin"]].describe(include="all")


,sched_flight_duration_minutes,departure_dow,departure_hour,fare_class,fare_class_ord,is_premium_cabin
count,574830.000000,574830.000000,574830.000000,574830,574830.0,574830.000000
unique,NaN,NaN,NaN,3,<NA>,NaN
top,NaN,NaN,NaN,Economy,<NA>,NaN
freq,NaN,NaN,NaN,506070,<NA>,NaN
mean,128.569551,2.951081,9.735158,NaN,0.222193,0.119618
std,107.191916,2.064185,3.583419,NaN,0.614795,0.324515
min,25.000000,0.000000,0.000000,NaN,0.0,0.000000
25%,55.000000,1.000000,7.000000,NaN,0.0,0.000000
50%,100.000000,3.000000,9.000000,NaN,0.0,0.000000
75%,155.000000,5.000000,12.000000,NaN,0.0,0.000000


In [9]:
# Join ticket numbers -> booking refs
tickets = tables["tickets"][["ticket_no","book_ref"]].copy()
tf_tk = tf_fl.merge(tickets, on="ticket_no", how="inner", validate="many_to_one")

# Attach booking-level fields (book_date, total_amount) – y kept separate for leakage safety
bookings = tables["bookings"][["book_ref","book_date","total_amount"]].copy()
bookings["book_date"]    = to_utc(bookings["book_date"])
bookings["total_amount"] = pd.to_numeric(bookings["total_amount"], errors="coerce")

seg_master = tf_tk.merge(bookings, on="book_ref", how="inner", validate="many_to_one")

# Booking lead time (days) = scheduled_departure - book_date
seg_master["booking_lead_time_days"] = days_between(seg_master["book_date"], seg_master["scheduled_departure"])

# Save a small sample as evidence
seg_master.head(10).to_csv(TAB_DIR/"seg_master_sample.csv", index=False)
seg_master.shape


(574830, 19)

In [10]:
# Aggregate segment -> booking
agg = seg_master.groupby("book_ref").agg(
    n_segments=("ticket_no","count"),
    sum_sched_duration_min=("sched_flight_duration_minutes","sum"),
    avg_sched_duration_min=("sched_flight_duration_minutes","mean"),
    max_sched_duration_min=("sched_flight_duration_minutes","max"),
    share_premium_cabin=("is_premium_cabin","mean"),
    max_cabin_index=("fare_class_ord","max"),
    has_longhaul=("sched_flight_duration_minutes", lambda s: int((s>=240).any())),
    n_unique_routes=("route_code","nunique"),
    primary_route_code=("route_code", lambda s: s.mode().iat[0] if not s.mode().empty else np.nan),
    avg_booking_lead_days=("booking_lead_time_days","mean"),
).reset_index()

# Attach target
booking_model_df = agg.merge(bookings[["book_ref","total_amount"]], on="book_ref", how="left", validate="one_to_one")
booking_model_df["log_total_amount"] = np.log(booking_model_df["total_amount"].astype(float))

# Persist artifacts
booking_model_df.to_parquet(TAB_DIR / "booking_model_df.parquet", index=False)
booking_model_df.head(20).to_csv(TAB_DIR/"booking_model_df_preview.csv", index=False)

# Quick summary for the report
num_cols = booking_model_df.select_dtypes(include=[np.number]).columns
summary = booking_model_df[num_cols].describe().T
summary.to_csv(TAB_DIR/"table_3_5_summary.csv")
booking_model_df.shape


(170227, 13)

 
**Next:** open `03_transformation_and_split.ipynb` (Step 4.1–4.3).
- We will apply transformations if needed, then create the train/test split and baseline pipelines.
- Modeling target: `log_total_amount`.
- Keep only features available **before departure** (no leakage).
